In [2]:
from unsloth import FastLanguageModel
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
import csv

/vol/bitbucket/kza23/msc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
DIALECT_NAME = "db_dialect"

csv.register_dialect(
    DIALECT_NAME,
    delimiter=",",
    quoting=csv.QUOTE_NONE,
    escapechar="\\",
)

In [4]:
max_seq_length = 20  # Choose any! We auto support RoPE Scaling internally!
dtype = (
    None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
)
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA A30 MIG 2g.12gb. Max memory: 11.688 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
dataset_vulnerable = load_dataset(
    "msc-smart-contract-audition/vulnerable-functions-base",
    split="train",
    escapechar="\\",
)

In [97]:
query_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
The below vulnerability audit contains an audit name, a description of a vulnerability and code blocks which contain the vulnerability.
You need to improve the description based on the information provided.
1. Expand the description in such a way that it is very comprehensive and easy to understand.
2. Do not dumb it down, still use technical terms where necessary.
3. Do not include the code blocks, but you could still use inline code for your explanation. Surround the inline code using single backticks.
4. In your reponse do not quote the title of the vulnerability.
5. Do not make up any information about the vulnerability that is not in the provided context.
6. Do not add your solution or mitigation to the issue.
<|start_header_id|>user<|end_header_id|>
Vulnerability title: {}
Vulnerability description:
{}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
Improved description:
"""


In [98]:
def apply_template(row):
    return query_template.format(
        row["name"],
        row["description"].replace("\\n", "\n"),
    )

# Convert to pandas DataFrame
df_vulnerable = dataset_vulnerable.to_pandas()
queries = df_vulnerable.apply(apply_template, axis=1)
queries = queries.drop(queries.index[534])

In [99]:
print(queries[188])


<|begin_of_text|><|start_header_id|>system<|end_header_id|>
The below vulnerability audit contains an audit name, a description of a vulnerability and code blocks which contain the vulnerability.
You need to improve the description based on the information provided.
1. Expand the description in such a way that it is very comprehensive and easy to understand.
2. Do not dumb it down, still use technical terms where necessary.
3. Do not include the code blocks, but you could still use inline code for your explanation. Surround the inline code using single backticks.
4. In your reponse do not quote the title of the vulnerability.
5. Do not make up any information about the vulnerability that is not in the provided context.
6. Do not add your solution or mitigation to the issue.
<|start_header_id|>user<|end_header_id|>
Vulnerability title: If the vault's side token balance is 0 or a tiny amount, then most if not all IG Bear trades will revert due to incorrect check of computation error dur

In [100]:
with open("vulnerability_descriptions.csv", "w", newline='') as f:
    writer = csv.writer(f, dialect=DIALECT_NAME)

    # Write the header row
    writer.writerow(["description"])

    # Process each query and write the result to the file
    # for query in tqdm(queries):
    for idx, query in enumerate(queries[:10]):
        inputs = tokenizer(query, return_tensors="pt", truncation=True).to("cuda")
        output_tokens = model.generate(
            **inputs, max_new_tokens=512, pad_token_id=tokenizer.pad_token_id
        )
        decoded_output = tokenizer.decode(
            output_tokens[0], skip_special_tokens=True, pad_token_id=tokenizer.pad_token_id
        )
        description = decoded_output.split("Improved description:\n")[1].strip()#.replace("\n", "\\n")
        # Write the description to the CSV file
        # writer.writerow([description])
        print("Original: ", df_vulnerable.iloc[idx]["description"])
        print("Imrpoved: ")
        print(description)
        print("====")
        print()

Original:  When liquidating a position, the liquidator is required to pay premium to Lender, which is accumulated in sharingProfitTokenAmts together with Lender's profit and paid to Lender in `_shareProfitsAndRepayAllDebts()`.\n```\n        (\n        netPnLE36,\n             lenderProfitUSDValueE36,\n                 borrowTotalUSDValueE36,\n                     positionOpenUSDValueE36,\n                      sharingProfitTokenAmts ) = calcProfitInfo(_positionManager, _user, _posId);\n        // 2. add liquidation premium to the shared profit amounts\n                            uint lenderLiquidatationPremiumBPS = IConfig(config).lenderLiquidatePremiumBPS();\n                         for (uint i; i < sharingProfitTokenAmts.length; ) {\n                    sharingProfitTokenAmts[i] +=\n                (pos.openTokenInfos[i].borrowAmt * lenderLiquidatationPremiumBPS) / BPS;\n        unchecked {\n        ++i;\n        }\n        }\n```\n\nHowever, if netPnLE36 <= 0, `_shareProfitsAndRep